# PEP Benchmarking - Guardian Dataset Both Algorithms

This notebook executes the PEP Benchmarking challenges on the Guardian Dataset for PEP Extraction Pipelines using automated algorithms for both Q-wave onset and B-point extraction.

## Setup and Helper Functions

In [ ]:
import json
from itertools import product
from pathlib import Path
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from fau_colors import cmaps, register_fausans_font
from IPython.display import Markdown
from tqdm.auto import tqdm

from pepbench.algorithms.ecg import (
    QPeakExtractionForounzafar2018,
    QPeakExtractionMartinez2004Neurokit,
    QPeakExtractionSciPyFindPeaksNeurokit,
    QPeakExtractionVanLien2013,
)
from pepbench.algorithms.heartbeat_segmentation import HeartbeatSegmentationNeurokit
from pepbench.algorithms.icg import (
    BPointExtractionArbol2017IsoelectricCrossings,
    BPointExtractionArbol2017SecondDerivative,
    BPointExtractionArbol2017ThirdDerivative,
    BPointExtractionDebski1993SecondDerivative,
    BPointExtractionDrost2022,
    BPointExtractionForouzanfar2018,
    BPointExtractionLozano2007LinearRegression,
    BPointExtractionLozano2007QuadraticRegression,
    BPointExtractionSherwood1990,
    BPointExtractionStern1985,
)
from pepbench.algorithms.outlier_correction import (
    OutlierCorrectionDummy,
    OutlierCorrectionForouzanfar2018,
    OutlierCorrectionLinearInterpolation,
)
from pepbench.datasets import GuardianDataset
from pepbench.evaluation import PepEvaluationChallenge
from pepbench.pipelines import PepExtractionPipeline

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
root_path = Path("../../")

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(guardian_base_path)

In [ ]:
result_path = root_path.joinpath("results/guardian_dataset_both_algorithms")
result_path.mkdir(exist_ok=True, parents=True)
result_path.resolve()

In [ ]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
dataset_guardian

## Initialize Algorithms

### Heartbeat Segmentation

In [ ]:
heartbeat_segmentation_algo = HeartbeatSegmentationNeurokit()

### ECG - Q-Wave Onset

In [ ]:
q_wave_algo_dict = {
    "martinez2004": QPeakExtractionMartinez2004Neurokit(),
    "forounzafar2018": QPeakExtractionForounzafar2018(),
    "scipy-findpeaks": QPeakExtractionSciPyFindPeaksNeurokit(),
}
q_wave_algo_dict.update(
    **{f"vanlien2013-{i}-ms": QPeakExtractionVanLien2013(time_interval_ms=i) for i in np.arange(32, 44, 2)}
)
q_wave_algos = list(q_wave_algo_dict.items())

print("Available Q-wave Onset algorithms:")
pprint(q_wave_algo_dict)

### ICG - B-Point

In [ ]:
b_point_algo_dict = {
    "stern1985": BPointExtractionStern1985(),
    "sherwood1990": BPointExtractionSherwood1990(),
    "debski1993-second-derivative": BPointExtractionDebski1993SecondDerivative(),
    "lozano2007-linear-regression": BPointExtractionLozano2007LinearRegression(),
    "lozano2007-quadratic-regression": BPointExtractionLozano2007QuadraticRegression(),
    "arbol2017-isoelectric-crossings": BPointExtractionArbol2017IsoelectricCrossings(),
    "arbol2017-second-derivative": BPointExtractionArbol2017SecondDerivative(),
    "arbol2017-third-derivative": BPointExtractionArbol2017ThirdDerivative(),
    "forounzafar2018": BPointExtractionForouzanfar2018(),
    "drost2022": BPointExtractionDrost2022(),
}
b_point_algos = list(b_point_algo_dict.items())

print("Available B-point algorithms:")
pprint(b_point_algo_dict)

### Outlier Correction

In [ ]:
outlier_correction_dict = {
    "none": OutlierCorrectionDummy(),
    "linear-interpolation": OutlierCorrectionLinearInterpolation(),
    "forouzanfar2018": OutlierCorrectionForouzanfar2018(),
}
outlier_correction_algos = list(outlier_correction_dict.items())

print("Available Outlier Correction algorithms:")
pprint(outlier_correction_dict)

In [ ]:
algo_combinations = list(product(q_wave_algos, b_point_algos, outlier_correction_algos))

In [ ]:
big_result_dict = {}

for algo_combi in tqdm(algo_combinations):
    algo_combi_names = [key[0] for key in algo_combi]

    filename_stub = "pep_results_guardian_" + "_".join(algo_combi_names)

    # check if exists
    if len(list(result_path.glob(f"{filename_stub}*"))) != 0:
        display(Markdown(f"Algrotihm combination {algo_combi_names} already challenged. Skipping..."))
        continue

    display(Markdown(f"Running algorithm combination {algo_combi_names}"))
    pep_challenge = PepEvaluationChallenge(dataset=dataset_guardian, validate_kwargs={"n_jobs": -1})

    pipeline = PepExtractionPipeline(
        heartbeat_segmentation_algo=heartbeat_segmentation_algo,
        q_wave_algo=algo_combi[0][1],
        b_point_algo=algo_combi[1][1],
        outlier_correction_algo=algo_combi[2][1],
        handle_negative_pep="nan",
        handle_missing_events="ignore",
    )
    pep_challenge.run(pipeline)
    pep_challenge.results_as_df()
    pep_challenge.save_results(result_path, filename_stub)

    big_result_dict[tuple(algo_combi_names)] = pep_challenge